<a href="https://colab.research.google.com/github/otmaniamine/meteo-station/blob/main/mini_project_temperature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

libraries

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

dispaly the data and normlise it

In [ ]:
data = pd.read_csv('/content/jena_climate_2009_2016.csv')  # Charger un fichier CSV
print(data)

                  Date Time  p (mbar)  T (degC)  Tpot (K)  Tdew (degC)  \
0       01.01.2009 00:10:00    996.52     -8.02    265.40        -8.90   
1       01.01.2009 00:20:00    996.57     -8.41    265.01        -9.28   
2       01.01.2009 00:30:00    996.53     -8.51    264.91        -9.31   
3       01.01.2009 00:40:00    996.51     -8.31    265.12        -9.07   
4       01.01.2009 00:50:00    996.51     -8.27    265.15        -9.04   
...                     ...       ...       ...       ...          ...   
420546  31.12.2016 23:20:00   1000.07     -4.05    269.10        -8.13   
420547  31.12.2016 23:30:00    999.93     -3.35    269.81        -8.06   
420548  31.12.2016 23:40:00    999.82     -3.16    270.01        -8.21   
420549  31.12.2016 23:50:00    999.81     -4.23    268.94        -8.53   
420550  01.01.2017 00:00:00    999.82     -4.82    268.36        -8.42   

        rh (%)  VPmax (mbar)  VPact (mbar)  VPdef (mbar)  sh (g/kg)  \
0        93.30          3.33          3.

J'ai modifié votre approche de dévision des données

In [ ]:
def signal_segmentation(dt, steps, ahead):
    x, y = [], []
    for i in range(steps, dt.shape[0] - ahead):
        x.append(dt[i-steps:i, :])
        y.append(dt[i:i+ahead,-1])
    return np.array(x), np.array(y)

# Load and prepare data
sf = data['T (degC)'].values.reshape(-1, 1)
train_size = int(len(sf) * 0.75)
train, test = sf[:train_size, :], sf[train_size:, :]

sequence_length = 144  # I defined the sequence length
ahead = 1  # Forecasting one step ahead (un pas en avant)
# Segmenting data
X_train, y_train = signal_segmentation(train, sequence_length, ahead)
X_test, y_test = signal_segmentation(test, sequence_length, ahead)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(315268, 144, 1)
(315268, 1)
(104993, 144, 1)
(104993, 1)


In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1] ))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1]))
scaler = MinMaxScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
# Reshape input
X_train = np.reshape(X_train, (X_train.shape[0], sequence_length, 1))
X_test = np.reshape(X_test, (X_test.shape[0], sequence_length, 1))
# Reshape output
y_train = np.reshape(y_train, (y_train.shape[0], ahead))
y_test = np.reshape(y_test, (y_test.shape[0], ahead))
print(X_train.shape)
print(y_train.shape)
print(X_train.shape)
print(y_test.shape)

(315268, 144, 1)
(315268, 1)
(315268, 144, 1)
(104993, 1)


LSTM model

In [ ]:
model = Sequential([
    LSTM(64, return_sequences=False, input_shape=(144, 1)),  # Increased units
    Dense(8, activation='relu'),
    Dense(1, activation='linear'),
])
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                        │ (None, 64)                  │          16,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 8)                   │             520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │               9 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 17,425 (68.07 KB)

 Trainable params: 17,425 (68.07 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
# Increase epochs during training
model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2)

Epoch 1/50
3941/3941 ━━━━━━━━━━━━━━━━━━━━ 36s 9ms/step - loss: 31.6298 - mae: 3.2647 - val_loss: 0.2258 - val_mae: 0.3370
Epoch 2/50
3941/3941 ━━━━━━━━━━━━━━━━━━━━ 33s 8ms/step - loss: 0.2453 - mae: 0.3541 - val_loss: 0.2424 - val_mae: 0.3968
Epoch 3/50
3941/3941 ━━━━━━━━━━━━━━━━━━━━ 41s 8ms/step - loss: 0.1423 - mae: 0.2685 - val_loss: 0.0726 - val_mae: 0.1753
Epoch 4/50
3941/3941 ━━━━━━━━━━━━━━━━━━━━ 42s 9ms/step - loss: 0.0924 - mae: 0.2133 - val_loss: 0.0516 - val_mae: 0.1518
Epoch 5/50
3941/3941 ━━━━━━━━━━━━━━━━━━━━ 32s 8ms/step - loss: 0.0640 - mae: 0.1769 - val_loss: 0.0427 - val_mae: 0.1400
Epoch 6/50
3941/3941 ━━━━━━━━━━━━━━━━━━━━ 41s 8ms/step - loss: 0.0571 - mae: 0.1667 - val_loss: 0.0488 - val_mae: 0.1593
Epoch 7/50
3941/3941 ━━━━━━━━━━━━━━━━━━━━ 42s 8ms/step - loss: 0.0655 - mae: 0.1597 - val_loss: 0.0761 - val_mae: 0.2122
Epoch 8/50
3941/3941 ━━━━━━━━━━━━━━━━━━━━ 41s 9ms/step - loss: 0.0558 - mae: 0.1637 - val_loss: 0.0685 - val_mae: 0.2107
Epoch 9/50
3941/3941 ━━━━━━━━━━

In [ ]:
model.save('temperature_prediction_model.h5')

In [ ]:
test_predictions = model.predict(X_test)
test_predictions = test_predictions.flatten()
y_test = y_test.flatten()
train_results = pd.DataFrame(data={'Train Predictions':test_predictions, 'Actuals':y_test})
train_results

3282/3282 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step


,Train Predictions,Actuals
0,1.309051,1.38
1,1.385301,1.40
2,1.379261,1.42
3,1.408728,1.49
4,1.500631,1.57
...,...,...
104988,-3.764708,-3.93
104989,-4.139677,-4.05
104990,-4.127889,-3.35
104991,-3.062613,-3.16


comparison of actuals to predictions values

In [ ]:
# Calculate MAE
mae = mean_absolute_error(y_test, test_predictions)
print(f"Mean Absolute Error (MAE): {mae:.2f}")
# Calculate MSE
mse = mean_squared_error(y_test, test_predictions)
print(f"Mean Squared Error (MSE): {mse:.2f}")
# Calculate RMSE
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
# Calculate R² score
r2 = r2_score(y_test, test_predictions)
print(f"R² Score: {r2:.2f}")

Mean Absolute Error (MAE): 0.13
Mean Squared Error (MSE): 0.04
Root Mean Squared Error (RMSE): 0.20
R² Score: 1.00


test sur la base de donnee de notre mini-projet

importe data from google sheet

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# TODO: Read the TP4.csv dataset file using the read_csv function (use a semicolon separator)
df = pd.read_csv('/content/dataset TP4.csv', sep=';')

# Convert columns (except the first one) to numeric, coercing errors to NaN
for col in df.columns[1:]:
    df[col] = pd.to_numeric(df[col], errors='coerce')


In [ ]:
print(df.isna().sum())
df = df.dropna(subset=["Temperature ambiante", "vitesse de vent ","Humidite"])
print(df.isna().sum())
df = df.reset_index(drop=True)

Temps                   0
Temperature ambiante    2
vitesse de vent         2
Humidite                2
dtype: int64
Temps                   0
Temperature ambiante    0
vitesse de vent         0
Humidite                0
dtype: int64


In [ ]:
import matplotlib.pyplot as plt

# Select the 'Temperature ambiante' column
df1 = df['Temperature ambiante']


In [ ]:
X, y = signal_segmentation(df1, 144, 1)

NameError: name 'signal_segmentation' is not defined

In [ ]:
predictions = model.predict(X)
results = pd.DataFrame(data={'Predictions': predictions.flatten(), 'Actuals': y.flatten()})
results

2158/2158 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step


,Predictions,Actuals
0,101.222862,12.0
1,101.192444,12.2
2,101.350540,12.4
3,101.682396,12.6
4,101.988007,12.7
...,...,...
69043,100.841919,8.9
69044,100.801170,9.1
69045,100.736610,9.0
69046,100.786636,8.5


In [ ]:
import gspread
from google.oauth2.service_account import Credentials
scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

creds = Credentials.from_service_account_file('credentials.json', scopes=scopes)
client = gspread.authorize(creds)
sheet_id = "15aYsguqDwRKMuUshvN8VqOC5XT-FEIwNMwwlNgYYgXo"
workbook = client.open_by_key(sheet_id)
sheet = workbook.worksheet("Sheet2")
data = sheet.get_all_records()
df = pd.DataFrame(data)
df = df[2:]
temperature = df['Temperature'].values.reshape(-1, 1)
print(temperature)

faire la prediction

In [ ]:
df['Date']+df['Time'] =pd.todatetime(df['Date']+' '+df['Time'])
df_resample  = df.set_index('Date'+'Time').resample('10T').mean()
df_resample.head()

export les resultas sur un notre fichier exel

In [ ]:
import gspread
from google.oauth2.service_account import Credentials
scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

creds = Credentials.from_service_account_file('credentials.json', scopes=scopes)
client = gspread.authorize(creds)
sheet_id = "1_3DtsUNf7wvkqfnMUjXgEwuCuM6L3NnsUgYfoQUVLDs"
workbook = client.open_by_key(sheet_id)
sheet = workbook.worksheet("sheet1")
sheet.update_cell(1, 1, "temperature")
num_predictions = temp_predict.shape[0]

# Prepare the data for updating multiple cells
cell_list = sheet.range(f'A2:A{num_predictions + 1}')  # Start from A2 to avoid overwriting "temperature"
for i, cell in enumerate(cell_list):
    # Convert the NumPy float32 to a regular Python float
    cell.value = float(temp_predict[i][0])

# Update the cells in one batch
sheet.update_cells(cell_list)

{'spreadsheetId': '1_3DtsUNf7wvkqfnMUjXgEwuCuM6L3NnsUgYfoQUVLDs',
 'updatedRange': 'sheet1!A2',
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}